In [1]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
import numpy as np
import pandas as pd

In [2]:
labels_path = "datasets/traffic_sign/labels.csv"
classes = pd.read_csv(labels_path)
class_names = list(classes['Name'])
num_classes = len(class_names)
data_dir = "datasets/traffic_sign/traffic_Data"

# Load the class labels
class_labels = pd.read_csv(labels_path)
class_names = list(class_labels['Name'])
num_classes = len(class_names)

# Data Preprocessing and Augmentation
batch_size = 32
image_size = (75, 75)  # Minimum input size required by InceptionV3

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    fill_mode='nearest',
    preprocessing_function=lambda x: tf.image.resize(x, image_size)
)

# Split the dataset into training and validation sets
train_datagen = datagen.flow_from_directory(
    data_dir+"/DATA",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_datagen = datagen.flow_from_directory(
    data_dir+"/TEST",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False,
    # subset='validation' 
)

Found 4170 images belonging to 58 classes.
Found 2003 images belonging to 58 classes.


In [6]:
import os
import shutil
from PIL import Image

original_data_dir = r"datasets\traffic_sign\traffic_Data\DATA"

In [ ]:
import os
import shutil
from PIL import Image

# Define the paths to the original dataset and augmented data directory
original_data_dir = r"datasets\traffic_sign\traffic_Data\DATA"
augmented_data_dir = r"datasets\traffic_sign\traffic_Data\DATA_AUG"

# Create the augmented data directory if it does not exist
os.makedirs(augmented_data_dir, exist_ok=True)

# Perform data augmentation and save the augmented images to the augmented data directory
for root, dirs, files in os.walk(original_data_dir):
    for file in files:
        image_path = os.path.join(root, file)
        image = Image.open(image_path)

        # Horizontal flip
        flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
        flipped_image.save(os.path.join(augmented_data_dir, "flipped_" + file))

        # Vertical flip
        flipped_image = image.transpose(Image.FLIP_TOP_BOTTOM)
        flipped_image.save(os.path.join(augmented_data_dir, "flipped_" + file))

        # Center crop
        width, height = image.size
        crop_size = min(width, height)
        left = (width - crop_size) // 2
        top = (height - crop_size) // 2
        right = left + crop_size
        bottom = top + crop_size
        cropped_image = image.crop((left, top, right, bottom))
        cropped_image.save(os.path.join(augmented_data_dir, "cropped_" + file))

In [7]:
# Merge the augmented data files with the original dataset
for root, dirs, files in os.walk(augmented_data_dir):
    for file in files:
        shutil.move(os.path.join(root, file), os.path.join(original_data_dir, file))

In [7]:
# Update the class labels and number of classes
labels_path = r"datasets\traffic_sign\labels.csv"
class_labels = pd.read_csv(labels_path)
# class_names = list(class_labels['Name'])
class_names = [str(i).zfill(3) for i in range(num_classes)]
num_classes = len(class_names)

# Update the training dataset
train_datagen = datagen.flow_from_directory(
    original_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

# Calculate the statistics of the dataset before augmentation
original_dataset_size = len(train_datagen.filenames)
original_num_classes = len(train_datagen.class_indices)
original_class_counts = train_datagen.classes

# Calculate the statistics of the dataset after augmentation
augmented_dataset_size = len(os.listdir(original_data_dir)) - original_num_classes + original_dataset_size
augmented_num_classes = len(class_labels)
augmented_class_counts = [len(os.listdir(original_data_dir+"\\"+class_name)) for class_name in class_names]

# Print the statistics of the dataset before and after augmentation
print("Statistics of the dataset before augmentation:")
print("Dataset size:", original_dataset_size)
print("Number of classes:", original_num_classes)
# print("Class counts:", original_class_counts)

print("\nStatistics of the dataset after augmentation:")
print("Dataset size:", augmented_dataset_size)
print("Number of classes:", augmented_num_classes)
# print("Class counts:", augmented_class_counts)


Found 4170 images belonging to 58 classes.
Statistics of the dataset before augmentation:
Dataset size: 4170
Number of classes: 58

Statistics of the dataset after augmentation:
Dataset size: 12510
Number of classes: 58
